In [1]:
# Parameters
cpu = 8
group_name = "Astro-TE_NN"
mem_gb = 10


In [2]:
import pandas as pd
import subprocess
import numpy as np

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name

_dir ='/data/dmg_result'

In [5]:
all_dmg = pd.read_csv(f"{_dir}/Male.CGN.DMG.csv", index_col =0)
all_dmg['celltype'] = all_dmg['celltype'].apply(lambda x: x.replace('/','').replace(' ','_'))

/var/tmp/ipykernel_74973/2186674015.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  all_dmg = pd.read_csv(f"{_dir}/Male.CGN.DMG.csv", index_col =0)


In [6]:
gene_meta = pd.read_csv(f"/data/ref/gencode.vM22.basic.annotation.gene.flat.tsv.gz", sep = '\t')
gene_meta.head()

,chrom,source,feature,start,end,score,strand,phase,gene_id,transcript_id,...,gene_name,transcript_type,transcript_status,transcript_name,exon_number,exon_id,level,mgi_id,havana_gene,tag
0,chr1,HAVANA,gene,3073253,3074322,.,+,.,ENSMUSG00000102693.1,NaN,...,4933401J01Rik,NaN,NaN,NaN,NaN,NaN,2,MGI:1918292,OTTMUSG00000049935.1,NaN
1,chr1,ENSEMBL,gene,3102016,3102125,.,+,.,ENSMUSG00000064842.1,NaN,...,Gm26206,NaN,NaN,NaN,NaN,NaN,3,MGI:5455983,NaN,NaN
2,chr1,HAVANA,gene,3205901,3671498,.,-,.,ENSMUSG00000051951.5,NaN,...,Xkr4,NaN,NaN,NaN,NaN,NaN,2,MGI:3528744,OTTMUSG00000026353.2,NaN
3,chr1,HAVANA,gene,3252757,3253236,.,+,.,ENSMUSG00000102851.1,NaN,...,Gm18956,NaN,NaN,NaN,NaN,NaN,1,MGI:5011141,OTTMUSG00000049958.1,pseudo_consens
4,chr1,HAVANA,gene,3365731,3368549,.,-,.,ENSMUSG00000103377.1,NaN,...,Gm37180,NaN,NaN,NaN,NaN,NaN,2,MGI:5610408,OTTMUSG00000049960.1,NaN


In [7]:
dmg =all_dmg[all_dmg['celltype'] == ct].copy()
dmg['gene_name'] = dmg.index.map(gene_meta.set_index('gene_id')['gene_name'].to_dict())
# change columns name 8wk to 2mo
dmg.columns = dmg.columns.str.replace('8wk', '2mo')
# add a column DMG, make corrected_pvalue < 0.05 and methylation_change > 0 as -1; corrected_pvalue < 0.05 and methylation_change < 0 as 1; the rest as 0
dmg['DMG'] = np.where((dmg['methylation_change'] > 0) & (dmg['pvalue'] < 0.05), -1, np.where((dmg['methylation_change'] < 0) & (dmg['pvalue'] < 0.05), 1, 0))
#dmg drop celltype, methylation_change, abs_change, trend
dmg = dmg.drop(columns = ['celltype', 'methylation_change', 'abs_change', 'trend'])
dmg.head()

,pvalue,corrected_pvalue,18mo,2mo,9mo,gene_length,DMG,gene_name
geneslop2k,,,,,,,,
ENSMUSG00000117337.1,1.264014e-05,0.004679,0.754929,0.653444,0.719705,1713,-1,Gm49964
ENSMUSG00000094483.2,7.946205e-08,0.000148,0.634706,0.521179,0.604175,8318,-1,Purb
ENSMUSG00000036430.8,1.013756e-06,0.000840,0.664198,0.545210,0.593113,1163,-1,Tbcc
ENSMUSG00000106103.1,3.029019e-06,0.001746,0.844819,0.734021,0.763767,1379,-1,Gm43081
ENSMUSG00000028675.12,4.924092e-08,0.000106,0.686385,0.568628,0.635950,2932,-1,Pnrc2


In [8]:
dmg.to_csv(f"{home_dir}/ml_input/{ct}/{ct}.mCG_genebody_gene.csv")